In [14]:
import requests
from bs4 import BeautifulSoup
import re
import os
import pandas as pd # Dùng để đọc và hiển thị file csv/tsv
from datetime import datetime, timedelta # Dùng để xử lý dữ liệu thời gian

In [15]:
url = 'https://www.worldometers.info/coronavirus/'
r = requests.get(url)
soup = BeautifulSoup(r.text, 'html.parser')

# Cào bảng

## Get columns

In [16]:
cols = ['Country, Other', 'Total Cases', 'New Cases', 'Total Deaths', 'New Deaths', 'Total Recovered', 'New Recovered', 'Active Cases', 'Serious, Critical',
        'Tot Cases/1M pop', 'Deaths/1M pop', 'Total Tests', 'Tests/1M pop', 'Population']
len(cols)        

14

## Get rows

Ý tưởng: 
- Ta sẽ cào bảng chính (`table`), bảng này có các dòng `rows` chưa bao gồm 2 dòng đầu (World) và cuối (Total) vì 2 dòng này chứa code html khác pattern của bảng chính ta đang tìm kiếm
- Cào riêng 2 dòng đầu & cuối và lưu lần lượt vào `row_0` và `row_last` rồi merge 2 dòng này vào các dòng hiện có
- Sau đó lặp qua từng dòng, tất cả thông tin trường của 1 dòng được lưu trong `t_row`, mỗi `t_row` này sau đó sẽ được append vào `table_data` để sau cùng hình thành 1 dataframe `df` chứa tất cả các dòng như trong trang web https://www.worldometers.info/coronavirus/

In [17]:
def get_table(which):
  '''which: today/yesterday/2 days ago'''
  table = soup.find('table', {"id": which})
  row_0 = table.tbody.find_all('tr', {'style': ''})[0]
  row_last = table.find('tbody', {'class': 'total_row_body body_world'})
  rows = table.tbody.find_all('tr', {'style': re.compile(r'(^$|^background)')})
  rows.insert(0, row_0)
  rows.append(row_last)
  table_data = []

  for row in rows:
    j=0
    t_row = {}
    for td in row.find_all("td")[1:]: # bỏ qua '#STT'
      if j == len(cols): # bỏ thẻ 'td' thừa
        break
      t_row[cols[j]] = td.text.replace('\n', '').strip()
      j+=1
    table_data.append(t_row)
  df = pd.DataFrame(table_data) # convert table_data to dataframe
  return df

Cách nhóm cào dữ liệu: cào cả 2 ngày (hôm qua, 2 hôm trước), việc này đảm bảo dữ liệu được lấy về (tức mỗi file csv trong thư mục Worldometer-data) chứa thông tin về dịch bệnh đầy đủ trong vòng 24h

In [18]:
which = ['main_table_countries_yesterday', # yesterday
         'main_table_countries_yesterday2'] # 2 days ago

for i in range(len(which)):
  d = (datetime.today() - timedelta(days=i)).strftime('%d-%m-%Y')  
  df = get_table(which[i])
  df.to_csv(f'./Worldometer-data/table_{d}_raw.csv')     

---

# Cào `data-continent` (châu lục) cho mỗi country để phục vụ cho quy trình trực quan hóa

- Trong đoạn mã html, mỗi nước đều được nhúng thẻ chứa **data_continent**, ta sẽ cào thêm thuộc tính này để thuận tiện cho pha trực quan hóa lúc sau. 
- Ý tượng cào tương tự như trên. Kết quả sau khi cào là `continent_df` có 2 cột: 1 cột quốc gia, 1 cột là châu lục quốc gia đó thuộc về

In [19]:
if not os.path.exists('./utils/continents.csv'):
  table = soup.find('table', {'id': 'main_table_countries_today'})
  row_0 = table.tbody.find_all('tr', {'style': ''})[0]
  row_last = table.find('tbody', {'class': 'total_row_body body_world'})
  rows = table.tbody.find_all('tr', {'style': re.compile(r'(^$|^background)')})
  rows.insert(0, row_0)
  rows.append(row_last)
  continent = {}

  for row in rows:
    row_info = row.find_all("td")[1:]
    country = row_info[0].text.replace('\n', '').strip()

    for td in row_info: 
      try:
        if td['data-continent'] != '': 
          continent[country] = td['data-continent']
          break
      except:
        continue
  continent

  continent_df = pd.DataFrame.from_dict(continent, orient='index', columns=['continent'])
  continent_df.to_csv('./utils/continents.csv')